In [21]:
"""
This Notebook is designed to re-write the XML files so that
non-existent meshes/*.ply files specified within the XML
file are deleted:
(<string name="filename" value="meshes/element_001-itu_brick.ply" />).
It simultaneously checks for whether the material id:
(<ref id="mat-itu_brick" name="bsdf" />)
are properly defined. E.g. we dont want id="mat-roof"
"""

import xml.etree.ElementTree as ET
import os
from os.path import join, dirname


def splitting_a_line(lll, uuid_incl='n'):
    lll = lll.replace('(', '')
    lll = lll.replace(')', '')
    lll = lll.replace('\n', '')
    lll = lll.split(',')
    # file format: (minLon,maxLat,maxLon,minLat),percent,idx_uuid
    if uuid_incl == 'y':
        minLon, maxLat, maxLon, minLat, perc, idx_uuid = [k for k in lll]
        return float(minLon), float(maxLat), float(maxLon), float(minLat), float(perc), idx_uuid
    else:
        minLon, maxLat, maxLon, minLat, perc = [float(k) for k in lll]
        return minLon, maxLat, maxLon, minLat, perc


BASE_PATH = '/home/yl826/res_plane/'  # probably /dev/shm/res_plane/
# BASE_PATH = '/Users/zeyuli/Desktop/Duke/0. Su23_Research/Blender_stuff/res_plane_missing'
XML_PATH = 'Bl_xml_files/'

# path containing all the fully-generated signal maps.
BASE_PATH_SIONNA = '/home/yl826/3DPathLoss/nc_raytracing/cm_512_Aug10_7e6_isoTx/'

In [22]:

idx_uuid_sig_map = set([f.split('_')[0] + '_' + f.split('_')[1]
                        for f in os.listdir(BASE_PATH_SIONNA)
                        if os.path.isfile(BASE_PATH_SIONNA + f)])

idx_uuid_fpath = join(BASE_PATH, 'step1.5.txt')
f_ptr = open(idx_uuid_fpath, 'r')
lines = f_ptr.readlines()
f_ptr.close()

idx_uuid_res_plane = set([splitting_a_line(line, uuid_incl='y')[-1] for line in lines])

# set of idx_uuid with XML files that can't produce sig map due to error.
bad_idx_uuid = idx_uuid_sig_map ^ idx_uuid_res_plane
print(len(bad_idx_uuid))

0


In [19]:
# temp_idx_uuid = '6274_6097b38d-e60c-402f-a516-aad0b488e004'
# temp_xml_file = join(BASE_PATH, XML_PATH, temp_idx_uuid, temp_idx_uuid) + '.xml'
# print(xmlfile)
print(bad_idx_uuid)

{'6231_80e39f72-8d27-4cc0-a4ff-4be83e986868', '840_80e6baee-8f3c-4df6-b180-c1ec7fbb6b63', '5164_9b428061-bfce-4681-b560-55c28c1673de', '4015_45aab618-ab6f-44a6-ae4a-056d16cc38ab', '6500_67805ca9-1a7e-4e38-b074-0485cede888c', '6292_03cf8cde-044f-45de-a3b1-7b48ab9b0b1b', '6170_54984b17-8cc5-468e-a0ed-b27be5a85c8d', '4024_005fee6b-a686-4e3c-9a4b-0711b706f2c7', '2599_35ee9842-cf05-4e63-84dc-d83569d6ef3c', '6168_3358c2c4-d392-44b0-bc29-430f8720b146', '6710_189f7567-2751-49e5-bdfe-ff58a9fd88a1', '5838_a39dd807-01eb-4d47-b6d4-fa27fbb74306', '5346_e20a7155-4663-45e0-9973-1e3de715bbae', '4678_371affc6-3b26-45c7-9107-e0682ff28b4b', '836_77ba6626-98f1-44ef-b209-14e05da9e7c7'}


In [20]:
def check_xml_and_write(idx_uuid):
    print('\n\nStart of idx_uuid')
    accepted_materials = {'itu_brick', 'itu_plasterboard', 'itu_concrete'}
    xmlfile = join(BASE_PATH, XML_PATH, idx_uuid, idx_uuid) + '.xml'
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    meshes_list = [f for f in os.listdir(join(BASE_PATH, XML_PATH, idx_uuid, 'meshes/'))]

    meshes_problem = []  # stores the indices in the tree where the mesh specified in XML doesn't exist as .ply
    xml_all_shapes = root.findall('shape')  # only iterating through the shapes and not other fields
    idx = 0
    changed_mat_name = False
    # finds all problematic shape indices
    for item in xml_all_shapes:
        inner_idx = 0
        for child in item:
            mesh_name = child.get('value')
            if inner_idx == 0:
                if mesh_name.split('/')[0] != 'meshes':
                    raise KeyError(str('mesh_name does not start with "meshes/"' + mesh_name))
                if mesh_name.split('/')[1] not in meshes_list:
                    meshes_problem.append(idx)
            if inner_idx == 2:
                old_id = str(child.get('id')).split('-')  # problematic ids will (hopefully) be: 'mat-wall' or 'mat-roof'
                if old_id[1] in accepted_materials:
                    continue
                print('old id: ' + old_id[0] + '-' + old_id[1])
                if old_id[1] != 'wall' and old_id[1] != 'roof':
                    return KeyError('mat name is neither wall nor roof')
                if old_id[1] == 'wall':
                    child.set('id', 'mat-itu_brick')
                if old_id[1] == 'roof':
                    child.set('id', 'mat-itu_brick')
                print('new id: ' + child.get('id'))
                changed_mat_name = True
            inner_idx += 1
        idx += 1
    if len(meshes_problem) == 0:
        print("Meshes don't have problems")
    if not changed_mat_name:
        print("Materials don't have any problems")
    if len(meshes_problem) == 0 and not changed_mat_name:
        print('XML is fine, returning')
        return
    ply_start_idx = 0
    for item in root:  # finds the index in tree wrt root where the first shape is
        if item.get('type') == 'ply':
            print(ply_start_idx)
            break
        ply_start_idx += 1

    meshes_problem = sorted(meshes_problem)
    # actually removes the problematic shape entries
    for diff_idx, prob_idx in enumerate(meshes_problem):
        for child in root[prob_idx + ply_start_idx - diff_idx]:
            print(child.get('value'))
            break
        root.remove(root[prob_idx + ply_start_idx - diff_idx])

    tree.write(join(BASE_PATH, XML_PATH, idx_uuid, idx_uuid) + '.xml')
    return


for temp_idx_uuid in bad_idx_uuid:
    check_xml_and_write(temp_idx_uuid)
# check_xml_and_write(idx_uuid='840_80e6baee-8f3c-4df6-b180-c1ec7fbb6b63')



Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't have problems
6


Start of idx_uuid
old id: mat-wall
new id: mat-itu_brick
old id: mat-roof
new id: mat-itu_brick
Meshes don't 

In [49]:
idx_res_plane = [int(f.split('_')[0])
                        for f in os.listdir(BASE_PATH_SIONNA)
                        if os.path.isfile(BASE_PATH_SIONNA + f)]


def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return freq
    # for key, value in freq.items():
    #     print("% d : % d" % (key, value))

freq_dict = CountFrequency(idx_res_plane)
# print(list(freq_dict.keys())[0:10])

for idx, key in enumerate(freq_dict.keys()):
    if freq_dict[key] != 4:
        print(freq_dict[key], key)